# Анализ базы данных сервиса для чтения книг по подписке    

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.  

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.    
Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.  


**Задача исследования:**  
Подготовить анализ базы данных, который в последствии поможет сформулировать ценностное предложение для нового продукта.
    
**Описание данных:**   
 
Таблица **`books`:**  
Содержит данные о книгах:   
`book_id` — идентификатор книги    
`author_id` — идентификатор автора    
`title` — название книги  
`num_pages` — количество страниц    
`publication_date` — дата публикации книги    
`publisher_id` — идентификатор издателя  

Таблица **`authors`:**  
Содержит данные об авторах:    
`author_id` — идентификатор автора  
`author` — имя автора   

Таблица **`publishers`:**  
Содержит данные об издательствах:  
`publisher_id` — идентификатор издательства    
`publisher` — название издательства  

Таблица **`ratings`:**  
Содержит данные о пользовательских оценках книг:  
`rating_id` — идентификатор оценки  
`book_id` — идентификатор книги  
`username` — имя пользователя, оставившего оценку   
`rating` — оценка книги  

Таблица **`reviews`:**  
Содержит данные о пользовательских обзорах:  
`review_id` — идентификатор обзора  
`book_id` — идентификатор книги  
`username` — имя автора обзора  
`text` — текст обзора  

**Схема базы данных:**   

<img src="https://pictures.s3.yandex.net/resources/scheme_1589269096.png" height="5"/></a> 

Настроим подключение к базе данных:

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Обзор данных

In [3]:
# выведем первые 5 строк таблицы books
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# выведем первые 5 строк таблицы authors
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# выведем первые 5 строк таблицы publishers
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
# выведем первые 5 строк таблицы ratings
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
# выведем первые 5 строк таблицы reviews
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Исследовательский анализ данных

### Посчитаем, сколько книг вышло после 1 января 2000 года:

In [8]:
query = '''SELECT COUNT(book_id) AS books_amount 
           FROM books 
           WHERE publication_date::date > '2000-01-01' 
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,books_amount
0,819


После 1 января 2000 года было выпущено 819 книг.

### Посчитаем количество обзоров и среднюю оценку для каждой книги:

In [9]:
query = '''SELECT b.title, 
                  b.book_id, 
                  COUNT(DISTINCT rev.review_id) AS review_amount, 
                  AVG(r.rating) AS avg_rating
           FROM books b
           LEFT JOIN reviews rev ON b.book_id = rev.book_id
           LEFT JOIN ratings r ON b.book_id = r.book_id
           GROUP BY b.book_id
           ORDER BY review_amount DESC, avg_rating DESC
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,book_id,review_amount,avg_rating
0,Twilight (Twilight #1),948,7,3.662500
1,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.414634
2,Harry Potter and the Chamber of Secrets (Harry...,299,6,4.287500
3,The Book Thief,656,6,4.264151
4,The Glass Castle,734,6,4.206897
...,...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,191,0,4.000000
996,Leonardo's Notebooks,387,0,4.000000
997,Essential Tales and Poems,221,0,4.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667


Больше всего обзоров пользователи написали на книгу Сумерки - всего 7 ревью.   
При этом, средний пользовательский рейтинг данной книги весьма невысок - 3.66 баллов из 5.   

На втором и третьем месте по количеству ревью - представители серии книг о Гарри Поттере - книга "Гарри Поттер и Узник Азкабана", а также книга "Гарри Поттер и Тайная комната" с количеством по 6 ревью у каждой. Рейтинг книг о Гарри Поттере выше, чем у книги "Сумерки", он составляет в среднем 4.3 - 4.4%.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц:

In [10]:
query = ''' WITH a AS (SELECT p.publisher_id, COUNT(DISTINCT b.book_id) as books_amount 
                       FROM publishers p
                       LEFT JOIN books b ON b.publisher_id = p.publisher_id
                       WHERE b.num_pages > 50
                       GROUP BY p.publisher_id
                       ORDER BY books_amount DESC
                       LIMIT 1)
            
            SELECT p.publisher, 
                   p.publisher_id, 
                   a.books_amount 
            FROM publishers p
            INNER JOIN a ON a.publisher_id = p.publisher_id
           '''
            
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,publisher_id,books_amount
0,Penguin Books,212,42


Наибольшее количество книг толще 50 страниц выпустило издательство Penguin Books (42 книги).

### Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками:

In [11]:
query = ''' WITH a AS (SELECT a.author, 
                              b.book_id,
                              COUNT(r.rating_id),
                              AVG(r.rating) AS avg_rating
                       FROM ratings r 
                       INNER JOIN books b ON b.book_id = r.book_id
                       INNER JOIN authors a ON a.author_id = b.author_id
                       GROUP BY a.author, b.book_id
                       HAVING COUNT(r.rating_id) >=50
                       ORDER BY avg_rating DESC)

                                                       
            SELECT author,
                   AVG(avg_rating)
            FROM a
            GROUP BY author
            ORDER BY AVG(avg_rating) DESC
            LIMIT 1
        '''
            
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Самую высокую среднюю оценку книг из авторов, которые пользователи оценили более 50 раз, имеет писательница Джоан Роулинг, автор всемирно известной серии романов о Гарри Поттере. Средняя оценка её книг - 4.28 баллов.

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок:

In [12]:
query = ''' WITH a AS (SELECT rev.username, 
                       COUNT(DISTINCT rev.review_id) 
                       FROM reviews rev
                       WHERE rev.username IN (SELECT r.username 
                                              FROM ratings r
                                              GROUP BY r.username
                                              HAVING COUNT(DISTINCT r.rating_id) > 48)
                                          
                        GROUP BY rev.username)
            
            SELECT AVG(count) AS avg_review_amount
            FROM a
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_review_amount
0,24.0


Среднее количество обзоров от пользователей, поставивших более 48 оценок - 24.

## Выводы

Исследование информации о книгах, издательствах, авторах, а также пользовательских обзоров книг были показало  следующие результаты:   
- После 1 января 2000 года было выпущено 819 книг  
- Больше всего обзоров пользователи написали на книгу Сумерки - 7 ревью  
- Наибольшее количество книг толще 50 страниц выпустило издательство Penguin Books (42 книги)  
- Самую высокую среднюю оценку из авторов, которых пользователи оценили более 50 раз, имеет писательница Джоан Роулинг, автор всемирно известной серии романов о Гарри Поттере. Средняя оценка её книг - 4.28 баллов  
- Среднее количество обзоров от пользователей, поставивших более 48 оценок - 24 обзора   

## Рекомендации

- Обратить внимание на крупное издательство Penguin Books, с которым можно заключить контракт на размещение книг в сервисе  
- Сделать акцент на предпочитаемые пользователями книги в жанрах фэнтези, роман, приключения  
- Увеличить количество книг для аудитории дети от 12 лет и подростки  
- Ввести бонусную/скидочную систему для пользователей, которые проявляют активность и пишут обзоры на книги  